# SCALib tutorial at CARDIS 2023

In this tutorial, we will perform a side-channel attack against the [ASCAD](https://github.com/ANSSI-FR/ASCAD) data set provided by the ANSSI. This is done using the side-channel analysis library [SCALib](https://github.com/simple-crypto/SCALib). This tutorial reproduces the results from [this paper](https://eprint.iacr.org/2021/817) for which the code is provided [here](https://github.com/cassiersg/ASCAD-5minutes). The traces can be downloaded from [this link (1.3 GB)](https://seafile.iaik.tugraz.at/f/1e3520a12fb34d6fbba0/).

### Imports
The next cell imports the required libraries. All the dependencies can be install with `pip install -r requirements.txt`.

In [ ]:
import copy
import collections
import functools as ft
import matplotlib.pyplot as plt

import h5py
import numpy as np
from scalib.metrics import SNR
import scalib.modeling
import scalib.attacks
import scalib.postprocessing
from tqdm import tqdm

### Settings for the profiling and attacks
Choosing parameters for the following experiments. The reduced dataset contains 5100 traces. As a result, `attack + profile` should be smaller than `5100`.

In [ ]:
class Settings:
    attacks = 100  # Number of single trace attacks
    profile = 5000  # Number of traces used in profiling
    poi = 512  # Number of POIs for each intermediate variables
    dim = 8  # Number of dimensions in LDA
    database = "./atmega8515-raw-traces_reduced.h5"  # Path to the database


settings = Settings()
assert(settings.attacks + settings.profile <= 5100)

### Helper functions
This next cell loads helper function to load the traces, instantiate AES Sbox, generates the labels of intermediate variables, etc. This is not really relevant for the tutorial.

In [ ]:
# Matplotlib configuration
FIGSIZE = (15, 4)
XLIM = [0, 250_000]
LW = 0.5
COLORS = ["b", "r", "c", "m", "y", "k"]

# number of bytes to attack
NBYTES = 14


def target_variables(byte):
    """variables that will be profiled"""
    return ["rin", "rout"] + [
        f"{base}_{byte}" for base in ("x0", "x1", "xrin", "yrout", "y0", "y1")
    ]


# fmt: off
SBOX = np.array(
    [
        0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB,
        0x76, 0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4,
        0x72, 0xC0, 0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71,
        0xD8, 0x31, 0x15, 0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2,
        0xEB, 0x27, 0xB2, 0x75, 0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6,
        0xB3, 0x29, 0xE3, 0x2F, 0x84, 0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB,
        0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF, 0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45,
        0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8, 0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5,
        0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2, 0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44,
        0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73, 0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A,
        0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB, 0xE0, 0x32, 0x3A, 0x0A, 0x49,
        0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79, 0xE7, 0xC8, 0x37, 0x6D,
        0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08, 0xBA, 0x78, 0x25,
        0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A, 0x70, 0x3E,
        0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E, 0xE1,
        0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
        0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB,
        0x16,
    ],
    dtype=np.uint32,
)

@ft.lru_cache(maxsize=None)
def load_database(settings):
    return h5py.File(settings.database, "r")

def var_labels(key, plaintext, masks, rin, rout):
    "Compute value of variables of interest based on ASCAD metadata."
    x0 = key ^ plaintext ^ masks
    x1 = masks
    xrin = ((key ^ plaintext).T ^ rin).T
    y0 = SBOX[key ^ plaintext].astype(np.uint16) ^ masks
    y1 = masks
    yrout = (SBOX[(key ^ plaintext).T].astype(np.uint16) ^ rout).T
    labels = {}
    for i in range(14):
        labels[f"k_{i}"] = key[:, i]
        labels[f"p_{i}"] = plaintext[:, i]
        labels[f"x0_{i}"] = x0[:, i]
        labels[f"x1_{i}"] = x1[:, i]
        labels[f"y0_{i}"] = y0[:, i]
        labels[f"y1_{i}"] = y1[:, i]
        labels[f"xrin_{i}"] = xrin[:, i]
        labels[f"yrout_{i}"] = yrout[:, i]
    labels[f"rout"] = rout[:]
    labels[f"rin"] = rin[:]
    return labels

@ft.lru_cache(maxsize=None)
def get_traces(settings, start, l):
    """Load traces and labels from ASCAD database."""
    I = np.arange(start, start + l)
    f_database = load_database(settings)
    traces = f_database["traces"][start : start + l, :].astype(np.int16)
    key = f_database["metadata"]["key"][I, 2:].astype(np.uint16)
    plaintext = f_database["metadata"]["plaintext"][I, 2:].astype(np.uint16)
    masks = f_database["metadata"]["masks"][I, 2:16].astype(np.uint16)
    rin = f_database["metadata"]["masks"][I, 16].astype(np.uint16)
    rout = f_database["metadata"]["masks"][I, 17].astype(np.uint16)
    labels = var_labels(key, plaintext, masks, rin, rout)
    return traces, labels

## Exercise 0: Visualize the traces
This cell plots a single trace. This works out of the box.

In [ ]:
traces, labels = get_traces(settings, start=0, l=1)

plt.figure(figsize=(15, 4))
plt.plot(traces[0, :], linewidth=LW)
plt.xlabel("Time")
plt.ylabel("Power Consumption")
plt.xlim([0, traces.shape[1]])
plt.show()

# Exercise 1a: Compute the SNR for all intermediate variables
The SNR of a variable $X$ is defined as:
$$
\mathrm{SNR}_X = \frac{\mathrm{Var}_{x\leftarrow X}(\mathrm{E}[L_x])}
                {\mathrm{E}_{x\leftarrow X}(\mathrm{Var}[L_x])
}$$
and is the variance between classes divided by the variance within the classes.
It can be efficiently computed with [scalib.metrics.SNR](https://scalib.readthedocs.io/en/stable/_modules/scalib/metrics/snr.html#SNR). In this exercise, you are required to fill the following code in order to assign `snrs[v]`  to the SNR value of the variable `v`. You can use the following cell in order to visualize the resulting SNR.


In [1]:
def compute_snr(settings):
    """Returns the SNR of the traces samples for each target variable."""

    snrs = dict()
    variables = [v for i in range(NBYTES) for v in target_variables(i)]
    traces, labels = get_traces(settings, start=0, l=settings.profile)

    for v in tqdm(variables, desc="SNR Variables"):
        # Compute the snr for variable v
        # TODO
        snrs[v] = np.array([0])

        # Avoid NaN in case of scope over-range
        np.nan_to_num(snrs[v], nan=0.0)
    return snrs


snrs = compute_snr(settings)

NameError: name 'settings' is not defined

In [ ]:
def plot_snr(snrs, variables):
    plt.figure(figsize=FIGSIZE)
    for color, var in zip(COLORS, variables):
        plt.plot(snrs[var], label=var, linewidth=LW, color=color)
    plt.xlim(XLIM)
    plt.xlabel("Time")
    plt.ylabel("SNR")
    plt.legend()
    plt.show()


plot_snr(snrs, target_variables(byte=0))

### Improvements
[scalib.metrics.SNR](https://scalib.readthedocs.io/en/stable/_modules/scalib/metrics/snr.html#SNR) has the ability to compute the SNR of multiple variables, which allows to save some performance overheads. In case the data-set is too large to fit in memory, the [scalib.metrics.SNR](https://scalib.readthedocs.io/en/stable/_modules/scalib/metrics/snr.html#SNR) can be updated in an incremental fashion by providing multiple chunks of data to `fit_u()`. As an extra exercise (not during the tutorial) you can:
1. Compute the SNR on all the variables with a single call to [scalib.metrics.SNR](https://scalib.readthedocs.io/en/stable/_modules/scalib/metrics/snr.html#SNR).
2. Provide the traces in multiple independent chunks to `fit_u()`.

# Exercise 1b: Select the Points-of-Interest (POIs)

In this exercise, you are asked to compute the POIs for each of the variables. The POIs for the variable `v` are the `settings.poi` indexes with the largest SNR. To compute the POIs, you can leverage [np.argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html#numpy-argsort) function.

In [ ]:
def compute_pois(settings, snrs):
    """Compute the POIs for all variables."""
    pois = dict()

    # Select POIs
    for v, snr in snrs.items():
        # Compute the POIs for the variable v.
        # TODO
        poi = np.array([0])
        
        pois[v] = poi
    return pois


pois = compute_pois(settings, snrs)

In [ ]:
def plot_pois(snrs, pois, variables):
    plt.figure(figsize=FIGSIZE)
    for color, var in zip(COLORS, variables):
        poi = pois[var]
        snr = snrs[var]
        plt.plot(snr, label=var, linewidth=LW, color=color)
        plt.scatter(poi, snr[poi], color=color, alpha=0.3, marker="x")
    plt.xlim(XLIM)
    plt.xlabel("Time")
    plt.ylabel("SNR and POIs")
    plt.legend()
    plt.show()


plot_pois(snrs, pois, target_variables(byte=0))

# Exercise 2: Build Gaussian templates with LDA
In the following, you are asked to build the Gaussian templates in order to derive the distribution of the leakage of each intermediate variables. Concretely, the leakage distribution is approximated with:
$$
 \mathsf{\hat{f}}(\mathbf{l} | x)= \frac{1}{\sqrt{(2\pi)^{p} \cdot |\mathbf{\Sigma} |}} \cdot \exp^{\frac{1}{2} (\mathbf{W} \cdot \mathbf{l} - \mathbf{\mu}_x)                     \mathbf{\Sigma}
                    ( \mathbf{W} \cdot \mathbf{l}-\mathbf{\mu}_x)'}
$$
where $\mathbf{W}$ is a projection matrix obtained with LDA. This enables to project the leakage from a space with `settings.poi` dimensions to a linear sub-space with `settings.dim` dimensions. This projection maximizes the separation between the classes. The Gaussian templates are then built in that subspace and the matrix $\mathbf{\Sigma}$ is the pooled covariance in that subspace.

In this exercise, you are asked to assign to `ldas[v]` a Gaussian template for variable `v`. Namely, you are asked to use [scalib.modeling.LDAClassifier](https://scalib.readthedocs.io/en/stable/source/api/scalib.modeling.LDAClassifier.html#scalib.modeling.LDAClassifier). The next cell enables to visualize the different classes in the linear subspace.

In [ ]:
def compute_templates(settings, pois):
    """Compute the POIs, LDA and gaussian template for all variables."""

    ldas = dict()
    variables = [v for i in range(NBYTES) for v in target_variables(i)]
    traces, labels = get_traces(settings, start=0, l=settings.profile)

    for v in tqdm(variables, desc="LDA Variables"):
        # Generate the Gaussian template for the variable v.
        # TODO
        
        ldas[v] = np.array([0])
    return ldas


ldas = compute_templates(settings, pois)

In [ ]:
def plot_lda(settings, pois, ldas, variable, values):
    traces, labels = get_traces(settings, start=0, l=settings.profile)
    projection = ldas[variable].lda.get_state()[0]

    plt.figure()
    for v in values:
        indexes = np.where(labels[variable] == v)[0]
        traces_v = traces[np.where(labels[variable] == v)[0], :]
        traces_v_projected = projection.T @ traces_v[:, pois[variable]].T
        plt.scatter(
            traces_v_projected[0, :],
            traces_v_projected[1, :],
            label=f"{variable}={v}",
            alpha=0.3,
        )

    plt.title("Distributions in linear subspace")
    plt.xlabel("First dimension")
    plt.ylabel("Second dimension")
    plt.legend()
    plt.show()


for variable in target_variables(byte=0):
    plot_lda(settings, pois, ldas, variable, np.random.randint(0, 256, 8))

### Improvements

The previous code can be improved in two ways:
1. When having to compute a LDA model for multiple variables, using sequentially multiple calls to `LDAClassifier` comes with overheads that can be avoided by leveraging [scalib.modeling.MultiLDA](https://scalib.readthedocs.io/en/stable/source/api/scalib.modeling.MultiLDA.html).
2. If the dataset is too large, it might not fit in memory at once. In such a case, it can be loaded in smaller chunks and the function `fit_u()` called for each of these chunks. 

# Exercise 3: A First key recovery

In the previous exercises, we have built templates for the shares within the implementation. In this exercise, we will run a SASCA that exploits the factor graph described below. To do so, you have to fill the TODOs in the following code.

The useful function are [scalib.attacks.FactorGraph](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.FactorGraph.html#scalib.attacks.FactorGraph) in order to generate a factor graph. Belief propagation can be ran thanks to [scalib.attacks.BPState](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.BPState.html#scalib.attacks.BPState) and the [bp_loopy](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.BPState.html#scalib.attacks.BPState.bp_loopy) function. The probabilities of the internal variables can be obtained from the LDA thanks to [predict_proba](https://scalib.readthedocs.io/en/stable/source/api/scalib.modeling.LDAClassifier.html#scalib.modeling.LDAClassifier.predict_proba) and then assigned to the factor graph variables thanks to [set_evidence](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.BPState.html#scalib.attacks.BPState.set_evidence). The produced distributions after running BP can be recovered thanks to [get_distribution](https://scalib.readthedocs.io/en/stable/source/api/scalib.attacks.BPState.html#scalib.attacks.BPState.get_distribution). We strongly encourage the have a look at the examples available in the documentation.

In [ ]:
SASCA_GRAPH = """
NC 256
TABLE sbox

VAR MULTI x
VAR MULTI x0
VAR MULTI x1
VAR MULTI xrin
VAR MULTI rin
VAR MULTI y
PUB MULTI p

VAR SINGLE k

PROPERTY x = p ^ k
PROPERTY x = x0 ^ x1
PROPERTY x = rin ^ xrin

"""

def attack(sasca_graph, traces, labels, ldas, pois):
    """Run a SASCA attack on the given traces
    Returns the true key and the byte-wise key distribution estimated by the attack.
    """
    # correct secret key
    secret_key = [labels[f"k_{i}"][0] for i in range(NBYTES)]
    
    # distribution for each of the key bytes
    key_distribution = []
    
    # Run a SASCA for each S-Box
    for i in range(NBYTES):

        # Init the sasca with the factor graph for a single trace
        # TODO
        graph = scalib.attacks.FactorGraph(sasca_graph, {"sbox": SBOX})

        # Set the labels for the plaintext byte
        # TODO
        
        # Set the initial distribution for target variables 'vs' if it is in the graph
        for v in target_variables(i):
            vs = v.split('_')[0]
            if vs in graph.vars():
                # Recover and assign the distribution of vs
                # TODO
                pass

        # Run 3 iterations of belief propagation
        # TODO

        # Get the distribution of the secret key
        # TODO
        distribution = np.ones(256)
        
        key_distribution.append(distribution)
        
    key_distribution = np.array(key_distribution)
    return secret_key, key_distribution

# Get the trace and label for the attack
traces, labels = get_traces(settings, start=settings.profile, l=1)
# Perform the attack
secret_key, key_distribution = attack(SASCA_GRAPH, traces,labels,ldas,pois)

print("Correct Key:", " ".join([f"0x{k:02x}" for k in secret_key] ))
print("Guessed Key:", " ".join([f"0x{np.argmax(k):02x}" for k in key_distribution] ))

# Exercise 4: Evaluate the resulting rank of the full correct key.

After performing a side-channel attack, an evaluator is usually interested in the remaining entropy of the key. That is, he wants to know the number of brute force an adversary needs before he will discover the correct key. This is call the key rank, and you will use SCALib to evaluate it in this exercise. To do so, you will use the function [calib.postprocessing.rank_accuracy](https://scalib.readthedocs.io/en/stable/source/api/scalib.postprocessing.rankestimation.html#module-scalib.postprocessing.rankestimation).

In [ ]:
def eval_rank(secret_key, key_distribution):
    """Returns the rank of the true key"""

    # Floor the key_distribution to avoid numerical issues
    key_distribution[np.where(key_distribution < 1e-100)] = 1e-100

    # Compute the rank of the
    # TODO

    return 2.0**64


print(f"Estimated rank 2**{np.log2(eval_rank(secret_key,key_distribution)):.2f}")

Putting all together, we are now able to evaluate the success rate of the adversary given an enumeration power. Concretely, the following code prints the success rate if the adversary does not perform enumeration and if the adversary is able to enumerate up to $2^{32}$ keys.

In [ ]:
def success_rate(ranks, min_rank=1):
    return np.mean(np.array(ranks) <= min_rank)


def attack_statistics(sasca_graph, settings, ldas, pois):
    ranks = []

    for i in tqdm(range(settings.attacks), desc="Evaluating the attack"):
        traces, labels = get_traces(settings, start=settings.profile + i, l=1)
        secret_key, key_distribution = attack(sasca_graph, traces, labels, ldas, pois)
        rank = eval_rank(secret_key, key_distribution)
        ranks.append(rank)

    print(f"Success rate (rank 1): {success_rate(ranks, min_rank=1)*100:.0f}%")
    print(f"Success rate (rank 2**32): {success_rate(ranks, min_rank=2**32)*100:.0f}%")


sasca_graph = SASCA_GRAPH
attack_statistics(sasca_graph, settings, ldas, pois)

# Exercise 5: Improving the attack

You may have noticed that the factor graph above does not include all the variables within the implementation. In order to improve the attack, and adversary can encode additional information in the factor graph. The variables `y0`, `y1`, `yrout`, `rout` can be added to the factor graph. This will improve the success rate of the attack.

In [ ]:
SASCA_GRAPH_IMPROVED = """
NC 256
TABLE sbox

VAR MULTI x
VAR MULTI x0
VAR MULTI x1
VAR MULTI xrin
VAR MULTI rin
VAR MULTI y
PUB MULTI p

VAR SINGLE k

PROPERTY x = p ^ k
PROPERTY x = x0 ^ x1
PROPERTY x = rin ^ xrin
"""
sasca_graph = SASCA_GRAPH_IMPROVED
attack_statistics(sasca_graph, settings, ldas, pois)